In [1]:
import pandas as pd
from scipy.signal import hilbert
import numpy as np
from matplotlib import pyplot as plt

from scipy.signal import find_peaks

%matplotlib qt

In [10]:
data = pd.read_pickle('~/munich/data/20240109/data.pkl')

In [3]:
data

,t,U,I
0,0.010000,3.999922,0.289357
1,0.020000,4.000033,0.289448
2,0.030000,3.999918,0.289547
3,0.040000,3.999930,0.290043
4,0.050000,3.999957,0.289708
...,...,...,...
5862413,58624.139798,3.999983,0.064023
5862414,58624.149798,3.999911,0.063024
5862415,58624.159798,4.000041,0.063489
5862416,58624.169798,3.999869,0.063192


In [11]:
data=data[(data['t']>1000) & (data['t']<20000)]

data['I_rel'] = data['I']-np.mean(data['I'])

In [12]:
signal = hilbert(data['I_rel'])

In [13]:
env = np.abs(signal)
phase = np.unwrap(np.angle(signal))

fit = np.polyval(np.polyfit(data['t'], phase, 1), data['t'])
phase_corr = phase-fit

In [14]:
spike_indicies, _ = find_peaks(np.sign(0.3) * data['U'],
                                   prominence=np.abs(0.3)*0.8, distance=1000)
spikes = data.iloc[spike_indicies]['t']

In [20]:
plt.close()

plt.figure()

plt.title('Hilbert transform')
plt.xlabel('time [s]')
plt.ylabel(r'$\phi$ (cumulative)')

plt.xlim(1800, 2030)
plt.ylim(107, 145)

plt.plot(data['t'], phase)
for pi in np.arange(0, 220, 2*np.pi):
    plt.axhline(pi, c='grey', ls=':')